# Функции потерь для задач классификации в PyTorch

Основной функцией потерь для задач классификации является *перекрестная энтропия* (cross-entropy):  

$$H(p,q)=-\sum_{y}p(y)\log q(y),$$
где $p$ – истинное вероятностное распределение, $q$ – вероятностное распределение, предсказанное моделью, $y$ – класс.  

В PyTorch существует несколько вариантов [функций потерь](https://pytorch.org/docs/stable/nn.html#loss-functions), связанных с перекрестной энтропией, основными из которых являются следующие:
- [BCELoss](#Binary-Cross-Entropy-(BCELoss)) – перекрестная энтропия для бинарной классификации;
- [CrossEntropyLoss](#Cross-Entropy-(CrossEntropyLoss)) – перекрестная энтропия для многоклассовой классификации;
- [NLLLoss](#Negative-Log-Likelihood-(NLLLoss)) – отрицательное логарифмическое правдоподобие (negative log likelihood);
- [BCEWithLogitsLoss](#Binary-Cross-Entropy-with-Logits-Loss-(BCEWithLogitsLoss)) – перекрестная энтропия для бинарной классификации с logits.

In [89]:
import numpy as np
import scipy.special
import torch 
import torch.nn as nn

precision = 8 # Точность после запятой

## Binary Cross Entropy (BCELoss)

*Бинарная перекрестная энтропия* (Binary Cross-Entropy, [BCELoss](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html)):  

$$L = -\frac{1}{l}\sum_{i=1}^{l}y_i\log{p(y_i)}+(1-y_i)\log{(1-p(y_i))},$$

где $y_i$ – истинный ответ, $y_i\in\{0,1\}$;  
$p(y_i)$ – предсказанная вероятность ответа $y_i=1$.  

Применяется для задач бинарной классификации.

#### Вычисление в NumPy
Обратите внимание на размерности массивов данных – это одномерные массивы.

In [90]:
# Два класса: [0, 1]
# Три примера
# Числа обозначают вероятности отнесения примера к классу 1
y_pred = np.array([0.1, 0.9, 0.2])
y_true = np.array([0.0, 1.0, 0.0])

In [91]:
# Binary Cross-Entropy
def BCE(y_pred, y_true):
    bce_loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return bce_loss

In [92]:
bce_loss_np = BCE(y_pred, y_true)
print(f'BCE loss (NumPy) = {bce_loss_np:.{precision}f}')

BCE loss (NumPy) = 0.14462153


#### Вычисление в PyTorch

In [93]:
y_pred_tensor = torch.from_numpy(y_pred)
y_true_tensor = torch.from_numpy(y_true)

In [94]:
bce_loss_fn = nn.BCELoss()
bce_loss_torch = bce_loss_fn(y_pred_tensor, y_true_tensor)
print(f'BCE loss (PyTorch) = {bce_loss_torch:.{precision}f}')

BCE loss (PyTorch) = 0.14462153


## Cross Entropy (CrossEntropyLoss)
Перекрестная энтропия в случае многоклассовой классификации (Categorical Cross-Entropy, Multiclass Cross-Entropy, [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)):  

$$L = -\frac{1}{l}\sum_{i=1}^{l}\sum_{c \in C}y_{ic}\log{p(y_{ic})},$$

где $l$ – количество примеров, $C$ – множество классов.

В PyTorch в функции `CrossEntropyLoss()` предсказанные значения считаются ненормализованными, поэтому перед вычислением перекрестной энтропии к ним применяется функция`Softmax`.

#### Вычисление в NumPy
На входе двумерные массивы: количество строк совпадает с количеством примеров, количество столбцов – с количеством классов.  
В каждой строке находятся вероятности принадлежности данного примера к классу, номер которого соответствует индексу столбца (от 0 до K–1).

In [95]:
# Три класса: [0, 1, 2]
# Четыре примера
y_pred = np.array([[0.8, 0.1, 0.1], [0.15, 0.8, 0.05], [0.05, 0.05, 0.9], [0.85, 0.1, 0.05]])
y_true = np.array([[1.0, 0.0, 0.0], [0.0,  1.0, 0.0],  [0.0,  0.0,  1.0], [1.0,  0.0, 0.0]])

In [96]:
# Применяем Softmax, как в PyTorch
y_pred_softmax = scipy.special.softmax(y_pred, axis=1)
y_pred_softmax

array([[0.5017132 , 0.2491434 , 0.2491434 ],
       [0.26175419, 0.50140083, 0.23684498],
       [0.2304335 , 0.2304335 , 0.53913301],
       [0.5203738 , 0.24580718, 0.23381902]])

In [130]:
def CrossEntropy(y_pred, y_true):
    total_loss = -np.sum(y_true * np.log(y_pred))
    num_of_samples = y_pred.shape[0]
    mean_loss = total_loss / num_of_samples
    return mean_loss

In [131]:
cross_entropy_loss_np = CrossEntropy(y_pred_softmax, y_true)
print(f'Cross Entropy loss (NumPy) = {cross_entropy_loss_np:.8f}')

Cross Entropy loss (NumPy) = 0.66276923


#### Бинарная классификация в NumPy (аналог BCELoss)
Для примера покажем вычисление бинарной перекрестной энтропии при помощи функции `CrossEntropy()`.  
Возьмем пример, который использовался выше в бинарной классификации.

In [99]:
# Два класса: [0, 1]
# Три примера
# Числа обозначают вероятности отнесения примера к классу 1
# y_pred = np.array([0.1, 0.9, 0.2])
# y_true = np.array([0.0, 1.0, 0.0])

# Преобразуем к двумерным массивам
y_pred = np.array([[0.9, 0.1], [0.1, 0.9], [0.8, 0.2]])
y_true = np.array([[1.0, 0.0], [0.0, 1.0], [1.0, 0.0]])

In [100]:
cross_entropy_loss_np = CrossEntropy(y_pred, y_true)
print(f'Cross Entropy loss (NumPy) = {cross_entropy_loss_np:.8f}')

Cross Entropy loss (NumPy) = 0.14462153


#### Вычисление в PyTorch
Вычисление многоклассовой перекрестной энтропии в PyTorch реализуется при помощи функции `CrossEntropyLoss()`.  
На вход функции поступает двумерный массив предсказанных вероятностей (строки – примеры, столбцы – классы) и одномерный массив истинных классов: значение в этом массиве обозначает класс (от 0 до K–1).

In [101]:
# Три класса: [0, 1, 2]
# Четыре примера
y_pred = np.array([[0.8, 0.1, 0.1], [0.15, 0.8, 0.05], [0.05, 0.05, 0.9], [0.85, 0.1, 0.05]])
y_true = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, 0.0]])

# Преобразуем двумерный массив y_true в одномерный
y_true = np.array([0, 1, 2, 0]).astype(np.int64) # номера классов из диапазона [0, K-1]

In [102]:
y_pred_tensor = torch.from_numpy(y_pred)
y_true_tensor = torch.from_numpy(y_true)

In [103]:
cross_entropy_loss_fn = nn.CrossEntropyLoss()
cross_entropy_loss_torch = cross_entropy_loss_fn(y_pred_tensor, y_true_tensor)
print(f'Cross Entropy loss (PyTorch) = {cross_entropy_loss_torch:.8f}')

Cross Entropy loss (PyTorch) = 0.66276923


#### Бинарная классификация в PyTorch (аналог BCELoss)
Также для примера продемонстрируем вычисление бинарной перекрестной энтропии при помощи функции PyTorch `CrossEntropyLoss()`.
Возьмем пример, который использовался выше в бинарной классификации.

In [104]:
# Два класса: [0, 1]
# Три примера
# Числа обозначают вероятности отнесения примера к классу 1
# y_pred = np.array([0.1, 0.9, 0.2])
# y_true = np.array([0.0, 1.0, 0.0])

# Преобразуем к двумерным массивам
y_pred = np.array([[0.9, 0.1], [0.1, 0.9], [0.8, 0.2]])
y_true = np.array([0, 1, 0]).astype(np.int64) # номера классов из диапазона [0, K-1]

In [105]:
y_pred_tensor = torch.from_numpy(y_pred)
y_true_tensor = torch.from_numpy(y_true) 

In [106]:
# Логарифмируем, чтобы компенсировать Softmax, встроенный в CrossEntropyLoss()
y_pred_log = torch.log(y_pred_tensor)

In [107]:
cross_entropy_loss_fn = nn.CrossEntropyLoss()
cross_entropy_loss_torch = cross_entropy_loss_fn(y_pred_log, y_true_tensor)
print(f'Cross Entropy loss (PyTorch) = {cross_entropy_loss_torch:.8f}')

Cross Entropy loss (PyTorch) = 0.14462153


## Negative Log Likelihood (NLLLoss)
*Отрицательное логарифмическое правдоподобие* (Negative Log Likelihood, [NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html)) – это другое название перекрестной энтропии.  
В PyTorch отрицательное логарифмическое правдоподобие реализуется при помощи функции `NLLLoss()`, но предполагается, что предсказанные ненормализованные значения $x_i$ преобразуются с помощью функции `LogSoftmax()`:

$$LogSoftmax(x_i)=\log\left(\frac{e^{x_i}}{\sum_{j}x_j}\right)$$

Поэтому в PyTorch перед использованием `NLLLoss()` ненормализованные выходы следует преобразовать с использованием функции `LogSoftmax()`.

#### Вычисление в NumPy

In [108]:
# Три класса: [0, 1, 2]
# Четыре примера
y_pred = np.array([[0.8, 0.1, 0.1], [0.15, 0.8, 0.05], [0.05, 0.05, 0.9], [0.85, 0.1, 0.05]])
y_true = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, 0.0]])

In [109]:
# Применяем Softmax
y_pred_softmax = scipy.special.softmax(y_pred, axis=1)
y_pred_softmax

array([[0.5017132 , 0.2491434 , 0.2491434 ],
       [0.26175419, 0.50140083, 0.23684498],
       [0.2304335 , 0.2304335 , 0.53913301],
       [0.5203738 , 0.24580718, 0.23381902]])

In [110]:
# Применяем логаримф - получается LogSoftmax
y_pred_log_softmax = np.log(y_pred_softmax)
y_pred_log_softmax

array([[-0.68972664, -1.38972664, -1.38972664],
       [-1.34034944, -0.69034944, -1.44034944],
       [-1.46779297, -1.46779297, -0.61779297],
       [-0.65320788, -1.40320788, -1.45320788]])

In [111]:
def NLLLoss(y_pred, y_true):
    # То же, что CrossEntropy, но без логарифма перед y_pred
    total_loss = -np.sum(y_true * y_pred)
    num_of_samples = y_pred.shape[0]
    mean_loss = total_loss / num_of_samples
    return mean_loss

In [112]:
nll_loss_np = NLLLoss(y_pred_log_softmax, y_true)
print(f'Negative Log Likelihood loss (NumPy) = {nll_loss_np:.8f}')

Negative Log Likelihood loss (NumPy) = 0.66276923


#### Вычисление в PyTorch

In [113]:
# Три класса: [0, 1, 2]
# Четыре примера
y_pred = np.array([[0.8, 0.1, 0.1], [0.15, 0.8, 0.05], [0.05, 0.05, 0.9], [0.85, 0.1, 0.05]])
y_true = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, 0.0]])

# Преобразуем двумерный массив y_true в одномерный
y_true = np.array([0, 1, 2, 0]).astype(np.int64) # номера классов из диапазона [0, K-1]

In [114]:
y_pred_tensor = torch.from_numpy(y_pred)
y_true_tensor = torch.from_numpy(y_true)

In [115]:
softmax = nn.LogSoftmax(dim=1)
y_pred_tensor = softmax(y_pred_tensor)

In [116]:
nll_loss_fn = nn.NLLLoss()
nll_loss_torch = nll_loss_fn(y_pred_tensor, y_true_tensor)
print(f'Negative Log Likelihood loss (PyTorch) = {nll_loss_torch:.8f}')

Negative Log Likelihood loss (PyTorch) = 0.66276923


## Binary Cross Entropy with Logits Loss (BCEWithLogitsLoss)
Функция [BCEWithLogitsLoss](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html) комбинирует `Sigmoid` и `BCELoss` в единый класс.  
Применяется для многозначной (*multilabel*) классификации.

#### Вычисление в NumPy

In [117]:
# Три класса: [0, 1, 2]
# Четыре примера, для каждого из которых может быть несколько классов
y_pred = np.array([[0.8, 0.1, 0.1], [0.15, 0.8, 0.05], [0.05, 0.05, 0.9], [0.85, 0.1, 0.05]])
y_true = np.array([[1.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0]])

In [118]:
def BCEWithLogitsLoss(y_pred, y_true, reduction='mean'):
    y_pred_sigmoid = scipy.special.expit(y_pred)
    
    loss = y_true * np.log(y_pred_sigmoid) + (1 - y_true) * np.log(1 - y_pred_sigmoid)
    
    if reduction == 'mean':
        bce_loss = -np.mean(loss)
    elif reduction == 'sum':
        bce_loss = -np.sum(loss)
    
    # Или (поскольку среднее находится по всем элементам матриц, а не по количеству объектов, как в Cross-Entropy):
    # sum_loss = -np.sum(loss)
    # if reduction == 'mean':
    #   bce_loss = sum_bce_loss / y_pred.size
    
    return bce_loss

In [119]:
bcewithlogits_loss_np = BCEWithLogitsLoss(y_pred, y_true, reduction='mean')
print(f'Binary Cross-Entropy with Logits Loss (NumPy) = {bcewithlogits_loss_np:.8f}')

Binary Cross-Entropy with Logits Loss (NumPy) = 0.58476716


Можно использовать ранее определенную функцию для Binary Cross Entropy (предварительно применив сигмоиду):

In [120]:
y_pred_sigmoid = scipy.special.expit(y_pred)
bce_loss_np = BCE(y_pred_sigmoid, y_true)
print(f'BCE loss (NumPy) = {bce_loss_np:.{precision}f}')

BCE loss (NumPy) = 0.58476716


#### Вычисление в PyTorch

In [121]:
# Три класса: [0, 1, 2]
# Четыре примера, для каждого из которых может быть несколько классов
y_pred = np.array([[0.8, 0.1, 0.1], [0.15, 0.8, 0.05], [0.05, 0.05, 0.9], [0.85, 0.1, 0.05]])
y_true = np.array([[1.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0]])

In [122]:
y_pred_tensor = torch.from_numpy(y_pred)
y_true_tensor = torch.from_numpy(y_true)

In [123]:
bcewithlogits_loss_fn = nn.BCEWithLogitsLoss(reduction='mean')
bcewithlogits_loss_torch = bcewithlogits_loss_fn(y_pred_tensor, y_true_tensor)
print(f'Binary Cross-Entropy with Logits Loss (PyTorch) = {bcewithlogits_loss_torch:.8f}')

Binary Cross-Entropy with Logits Loss (PyTorch) = 0.58476716


## Резюме
Обозначим `y` – ненормированный выход последнего слоя нейронной сети (например, выход модуля `Linear`).
1. Бинарная классификация:
    - `y` 🡒 `Sigmoid` 🡒 `BCELoss`
2. Многоклассовая классификация:
    - `y` 🡒 `LogSoftmax` 🡒 `NLLLoss`
    - `y` 🡒 `CrossEntropyLoss` (аналог: `y` 🡒 `LogSoftmax` 🡒 `NLLLoss`)
  
Если требуется на выходе сети явно иметь результаты `Softmax`, можно использовать следующую схему:  
- `y` 🡒 `Softmax` 🡒 `Log` 🡒 `NLLLoss`

3. Многозначная (*multilabel*) классификация:
    - `y` 🡒 `BCEWithLogitsLoss` (аналог: `y` 🡒 `Sigmoid` 🡒 `BCELoss`)